In [2]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv("amostra_aleatoria.csv")

In [5]:
df.head(3)

,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,...,OPN5_E,OPN6_E,OPN7_E,OPN8_E,OPN9_E,OPN10_E,dateload,country,lat_appx_lots_of_err,long_appx_lots_of_err
0,1.0,3.0,2.0,4.0,1.0,3.0,1.0,5.0,1.0,5.0,...,1993.0,1619.0,1667.0,2377.0,2925.0,916.0,2016-08-25 15:53:43,US,32.8807,-117.2359
1,4.0,1.0,5.0,1.0,4.0,1.0,5.0,2.0,4.0,1.0,...,3304.0,3456.0,2648.0,1744.0,2847.0,3648.0,2017-03-13 05:17:21,US,39.7887,-84.2163
2,2.0,2.0,5.0,1.0,5.0,1.0,4.0,2.0,5.0,1.0,...,1583.0,2843.0,10939.0,8427.0,2554.0,1541.0,2017-03-16 15:22:13,US,37.0505,-76.3523


In [6]:

# Suponha que você tem um DataFrame chamado 'df' com colunas 'latitude' e 'longitude'.
# Neste exemplo, usaremos 5 clusters (você pode ajustar o número conforme necessário).
num_clusters = 20

# Crie uma matriz de coordenadas a partir das colunas 'latitude' e 'longitude'.
coordinates = df[['lat_appx_lots_of_err', 'long_appx_lots_of_err']].values

# Aplique o algoritmo K-Means para agrupar as coordenadas em clusters.
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df['cluster'] = kmeans.fit_predict(coordinates)

# Divida os dados em grupos estratificados com base nos clusters gerados.
stratified_data = df.groupby('cluster', group_keys=False).apply(lambda x: x.sample(frac=1))  # Exemplo: amostra de 20% por cluster

# Agora você tem uma amostra estratificada em 'stratified_data'.


/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [7]:
stratified_data.cluster.value_counts()

cluster
1     69990
19    22309
15    18897
11    17958
2     15957
14    15107
13    14276
3     13815
7     12721
0     11451
6     10947
5      9639
4      8961
17     7999
18     6667
10     5611
16     5490
12     3679
8       887
9       595
Name: count, dtype: int64

In [8]:
pip install folium geopy

Note: you may need to restart the kernel to use updated packages.


In [15]:
import pandas as pd
import random

# Suponha que você tem o DataFrame 'df' com as colunas 'cluster' que representa os clusters e 'pessoas' que representa a quantidade de pessoas em cada cluster.
# Primeiro, encontre os clusters com pelo menos 10.000 pessoas.
clusters_mais_de_10000 = df.groupby('cluster').filter(lambda x: len(x) >= 10000)

# Em seguida, selecione aleatoriamente 10.000 pontos de cada um dos clusters restantes.
amostra_final = pd.DataFrame()
for cluster in clusters_mais_de_10000['cluster'].unique():
    cluster_data = clusters_mais_de_10000[clusters_mais_de_10000['cluster'] == cluster]
    if len(cluster_data) > 10000:
        selected_points = random.sample(cluster_data.index.tolist(), 1000)
        amostra_final = pd.concat([amostra_final, cluster_data.loc[selected_points]])

# 'amostra_final' agora contém uma amostra aleatória de 10.000 pontos de cada cluster com pelo menos 10.000 pessoas.


In [16]:
amostra_final.cluster.value_counts()

cluster
2     1000
11    1000
15    1000
3     1000
1     1000
14    1000
7     1000
0     1000
13    1000
6     1000
19    1000
Name: count, dtype: int64

In [17]:
len(amostra_final)

11000

In [36]:
# Crie um mapa centralizado nos Estados Unidos.
us_map = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

# Para cada cluster, encontre a coordenada representativa (poderia ser o centroide) e adicione um marcador circular proporcional ao tamanho do cluster.
for cluster in amostra_final['cluster'].unique():
    # Encontre a coordenada representativa do cluster (poderia ser o centroide).
    cluster_data = amostra_final[amostra_final['cluster'] == cluster]
    representative_coords = cluster_data[['lat_appx_lots_of_err', 'long_appx_lots_of_err']].mean()
    
    # Converta a coordenada para um nome de localização aproximado.
    geolocator = Nominatim(user_agent="cluster_representative")
    location = geolocator.reverse(representative_coords.to_list(), exactly_one=True)
    
    # Calcule o raio da bola proporcional ao tamanho do cluster.
    cluster_size = len(cluster_data)
    radius = cluster_size / 100  # Ajuste o fator de escala conforme necessário para controlar o tamanho das bolas.
    
    # Crie um marcador circular no mapa com o tamanho proporcional ao tamanho do cluster.
    folium.CircleMarker(
        location=representative_coords,
        radius=radius,
        popup=f"Cluster {cluster}<br>Tamanho: {cluster_size}<br>{location.address}",
        fill=True,
        color='blue',
        fill_color='blue'
    ).add_to(us_map)

# Salve o mapa em um arquivo HTML ou exiba-o no notebook.
us_map.save('us_clusters_map_with_circles.html')


/usr/local/lib/python3.11/site-packages/folium/utilities.py:59: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coords = (location[0], location[1])
/usr/local/lib/python3.11/site-packages/folium/utilities.py:59: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coords = (location[0], location[1])
/usr/local/lib/python3.11/site-packages/folium/utilities.py:59: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coords = (location[0], location[1]

In [18]:
amostra_final.to_csv("amostra_final.csv", sep=",")